In [1]:
import requests
import pandas as pd
from flatten_json import flatten
from utils.io import yaml_to_dict
import numpy as np
from typing import List, Tuple, Dict, Union
from itertools import compress
import pandas as pd
import re
import datetime as dt
from cleanco import basename
from json import dumps
import matplotlib.pyplot as plt
import seaborn as sns
import postproc
import matplotlib.colors as mc

id = "lgulden"
apikey = "4d1a4bc3be920e859b3862a25d3725d741028d42"
data_gov_api_key = "n4TEYUedn3STcnsxQi7JllzXZgfB6Dqg5pV2068E"
%load_ext jupyter_black

In [2]:
config = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/config_ccs_lda.yml"
)
term_list_dict = yaml_to_dict(config["search_term_list_path"])
law_list_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/ccs_laws.yml"
)

In [ ]:
authenticated_session = api_authenticate(
    config["authentication_endpoint"],
    config["lda_username"],
    config["lda_apikey"],
)
result = authenticated_session.get(
    config["filings_endpoint"],
    params={"filing_specific_lobbying_issues": f"{search_string}"},
)

In [6]:
def get_list_govt_entities(entity_endpoint: str, session: object):
    """Queries constants endpoint to get a standardized list of government entities"""
    govt_entities = session.get(entity_endpoint, timeout=60)
    entity_df = pd.DataFrame(govt_entities.json())
    entities = sorted([x.lower() for x in list(entity_df["name"])])
    return entities


def api_authenticate(
    authentication_endpoint,
    username,
    apikey,
    username_key="username",
    password_key="password",
):
    """logs in to the api using provided authentication endpoint and credentials"""
    authenticated_session = requests.Session()

    authenticated_session.post(
        authentication_endpoint,
        json={
            username_key: username,
            password_key: apikey,
        },
        timeout=60,
    )

    return authenticated_session


def terms_present(phrase, term_list, find_any=True):
    """utility function to see if terms in terms_list are present in a given phrase
    Args:
        phrase: phrase to be searched
        term_list: list of strings to be searched for within phrase
        find_any: boolean -- true if function should return true if any of the terms are in the phrase;
            false if the function should only return true if all terms are present
    Returns:
        int: 1 if one or more of the terms are present in phrase, 0 otherwise
    """
    if not isinstance(phrase, str):
        raise TypeError("phrase must be a string")

    if phrase is None:
        return 0
    n_present = 0
    for term in term_list:
        if not isinstance(term, str):
            raise TypeError(" all terms in term_list must be strings")
        if term.lower() in phrase.lower():
            if find_any:
                return 1
            n_present += 1
    if n_present == len(term_list):
        return 1
    return 0


def substitute(
    x: str,
    use_basename: bool = False,
    re_types: str = r"[^\w\s]",
    replace_str: str = "",
):
    """wrapper function for regular expression substitute funciton, linked with basename lib"""
    # use basename for company names
    if not isinstance(x, str):
        x = ""
    if use_basename:
        return basename(re.sub(re_types, replace_str, x))
    # don't use basename for general strings
    return re.sub(re_types, replace_str, x).rstrip().lstrip()


def parse_client_names(
    input_df, config, client_rename_col="client_rename"
) -> Tuple[pd.DataFrame, Dict[str, str]]:
    """bespoke function for parsing organization names from LDA queries
    Args:
        input_df: input pandas dataframe (read raw from files saved after api access)
        config: configuration dictionary containing details on how to handle names
        client_rename_column: name for new column in dataframe that is returned
    Returns:
        output_df: processed column with renamed clients and removed organziations
        client_name_rename_dict: dictionary used for renaming (contains original and new names)
    """
    output_df = input_df.copy(deep=True)

    starting_client_names = sorted(list(output_df.client_name.unique()))
    client_names = sorted(list(output_df.client_name.unique()))
    client_names = [
        substitute(company_name, use_basename=True) for company_name in client_names
    ]

    # take rightmost component of client name
    for term in config["take_terms_to_the_right_of_these_words"]:
        client_names = [x.split(term)[-1] for x in client_names]
    # take leftmost component of client name
    for term in config["take_terms_to_the_left_of_these_words"]:
        client_names = [x.split(term)[0] for x in client_names]
    # remove words
    for n in config["remove_these_phrases"]:
        client_names = [x.replace(n, "") for x in client_names]
    # get rid of double spaces
    client_names = [x.replace("  ", " ") for x in client_names]
    # trim spaces on ends of names
    client_names = [x.rstrip().lstrip() for x in client_names]

    # extract shorter, well-known names from longer names
    for co in config["use_these_name_subsets_for_organiztions"]:
        client_names = [co if co in x else x for x in client_names]

    # bespoke replacements and handling of mergers
    for key, value in config["replace_names_on_left_with_names_on_right"].items():
        client_names = [x.replace(key, value) for x in client_names]

    # make a renaming dictionary
    client_name_rename_dict = dict(zip(starting_client_names, client_names))

    # add the 'remove' companies to the rename dictionary
    remove_companies = config["remove_companies_containing_these_terms"]
    for x in client_name_rename_dict.keys():
        if terms_present(client_name_rename_dict[x], remove_companies, find_any=True):
            client_name_rename_dict[x] = "remove"

    # make new column with renames
    output_df[client_rename_col] = [
        client_name_rename_dict[x] for x in output_df.client_name
    ]
    output_df = output_df.loc[output_df[client_rename_col] != "remove"]

    return output_df, client_name_rename_dict


def get_smarties(
    row: Union[pd.Series, List[Union[bool, int]]], names: List[str]
) -> List[str]:
    if isinstance(row, pd.Series):
        return list(compress(names, row[names].values.tolist()))
    if isinstance(row, list):
        return list(compress(names, row))
    raise TypeError("get_smarties argument 'row' must be a Pandas Series or a list")


def get_latest_filings(
    df: pd.DataFrame, groupby_cols: List[str], date_col="filing_dt_posted"
):
    """get only the latest filing for a given lobbying firm, client, and quarter"""
    if df[date_col].dtype == str:
        df[date_col] = [dt.datetime.fromisoformat(d) for d in df[date_col]]

    df.sort_values(by=date_col, ascending=False, inplace=True)

    df = df.groupby(groupby_cols).first().reset_index()
    return df

In [206]:
config_info = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/config_ccs_lda.yml"
)
groupby_cols = [
    "filing_year",
    "filing_period",
    "client_id",
    "registrant_id",
    "activity_id",
]
entities = get_list_govt_entities(
    config_info["entity_endpoint"],
    session=api_authenticate(
        config_info["authentication_endpoint"],
        config_info["lda_username"],
        config_info["lda_apikey"],
    ),
)
remove_sector_descriptions = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sector_company_description_terms.yml"
)["remove"]

df_list = []
rename_dict = {}
for i in range(1, 19):  # 159):
    api_results_df = pd.read_csv(
        # f"/Volumes/Samsung_T5/data/lobbying/ccslaws/ccs_lda_filings_{i}.csv",
        f"/Volumes/Samsung_T5/data/lobbying/ccs/ccs_lda_filings_{i}.csv",
        index_col=[0],
        # parse_dates=["filing_dt_posted"],
        dtype={"filing_year": int},
        low_memory=False,
    )
    # remove unwanted filing types
    api_results_df = api_results_df.loc[
        [x[0] != "R" for x in api_results_df.filing_type]
    ]

    # parse company names, remove unwanted names, add to list
    df, this_rename_dict = parse_client_names(
        api_results_df, yaml_to_dict(config_info["organization_name_handling_path"])
    )

    # append the rename dictionary to the whole thing
    rename_dict = rename_dict | this_rename_dict
    # compress entities into a single string column and get rid of entity columns
    df["entities"] = df[entities].T.apply(lambda x: dumps(get_smarties(x, entities)))
    df.drop(entities, axis=1, inplace=True)
    df["clean_description"] = [
        substitute(d, use_basename=True) for d in df["description"]
    ]
    df["clean_client_general_description"] = [
        substitute(d, use_basename=False) for d in df["client_general_description"]
    ]
    df["client_rename"] = [
        "remove" if terms_present(x, remove_sector_descriptions) else n
        for x, n in zip(df.clean_client_general_description, df.client_rename)
    ]
    df["client_rename"] = [
        "remove" if terms_present(x, remove_sector_descriptions) else x
        for x in df.client_rename
    ]

    df = get_latest_filings(df, groupby_cols)
    df = df.loc[df.client_rename != "remove"]
    df_list.append(df)

ccs_df = pd.concat(df_list)
ccs_df = get_latest_filings(ccs_df, groupby_cols)
# fill in nans/nones with empty string for description and rename of client
ccs_df.clean_client_general_description = (
    ccs_df.clean_client_general_description.fillna("")
)
ccs_df.client_rename = ccs_df.client_rename.fillna("")
ccs_df["batch"] = "ccs description and/or ccs specific laws and bills"
ccs_df.to_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccs_compiled.csv")
#ccs_df["batch"] = "relevant laws"
#ccs_df.to_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccslaws_compiled.csv")

,filing_year,filing_period,client_id,registrant_id,activity_id,who_is_lobbying,dollars_spent_lobbying,filing_id,affiliated_organizations_present,convictions_present,...,client_ppb_state,client_ppb_country,general_issue_code,description,n_lobbyists_for_activity,which_congress,client_rename,entities,clean_description,clean_client_general_description
0,2001,mid_year,107351,5517,0,income and expenses are zero,0.0,3,False,False,...,ND,US,UTI,"FERC legislation - ISO, RTO & Transmission iss...",4,107,BASIN ELECTRIC POWER COOPERATIVE,[],FERC legislation ISO RTO Transmission issues P...,
1,2001,mid_year,133065,25328,0,income and expenses are zero,0.0,2,False,False,...,VA,US,ENG,clean coal technology,3,107,ESCRUB SYSTEMS,"[""house of representatives"", ""senate""]",clean coal technology,environmental enterprise
2,2001,mid_year,162103,62791,0,corporation lobbying for itself,20000.0,1,False,False,...,NC,US,AGR,Seeking earmarked funding for R&D on herbal ph...,1,107,RTI INTERNATIONAL,"[""house of representatives"", ""senate""]",Seeking earmarked funding for RD on herbal pha...,
3,2001,mid_year,162103,62791,1,corporation lobbying for itself,20000.0,0,False,False,...,NC,US,AGR,Seeking earmarked funding for R&D on herbal ph...,1,107,RTI INTERNATIONAL,"[""house of representatives"", ""senate""]",Seeking earmarked funding for RD on herbal pha...,
4,2001,mid_year,162103,62791,2,corporation lobbying for itself,20000.0,0,False,False,...,NC,US,AER,Advocating increased funding for aviation safe...,1,107,RTI INTERNATIONAL,"[""house of representatives"", ""senate""]",Advocating increased funding for aviation safe...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39079,2024,second_quarter,213641,401104734,0,hired lobbying firm,45000.0,13394,False,False,...,TX,US,SCI,Issues related to deployment of carbon capture...,2,118,HIGHLY INNOVATIVE FUELS,"[""energy, dept of"", ""house of representatives""...",Issues related to deployment of carbon capture...,Renewable energy
39080,2024,second_quarter,213641,401104734,1,hired lobbying firm,45000.0,13395,False,False,...,TX,US,TAX,Issues related to deployment of carbon capture...,2,118,HIGHLY INNOVATIVE FUELS,"[""energy, dept of"", ""house of representatives""...",Issues related to deployment of carbon capture...,Renewable energy
39081,2024,second_quarter,213641,401104734,2,hired lobbying firm,45000.0,13395,False,False,...,TX,US,NAT,Issues related to deployment of carbon capture...,2,118,HIGHLY INNOVATIVE FUELS,"[""energy, dept of"", ""house of representatives""...",Issues related to deployment of carbon capture...,Renewable energy
39082,2024,second_quarter,213641,401104734,3,hired lobbying firm,45000.0,13395,False,False,...,TX,US,CAW,Issues related to deployment of carbon capture...,2,118,HIGHLY INNOVATIVE FUELS,"[""energy, dept of"", ""house of representatives""...",Issues related to deployment of carbon capture...,Renewable energy


In [212]:
len(ccs_df)

235431

In [208]:
tmp_df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccs_compiled.csv")
law_df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccslaws_compiled.csv")
ccs_df = pd.concat([tmp_df, law_df])
ccs_df.sort_values(by=["batch"], ascending=True, inplace=True)
ccs_df.drop_duplicates(
    subset=["filing_uuid", "activity_id"], keep="first", inplace=True
)
ccs_df.to_csv("/Volumes/Samsung_T5/data/lobbying/ccs/ccsall_compiled.csv")

In [215]:
ccs_df.client_rename.nunique()

4931

In [216]:
for i, n in ccs_df.clean_client_general_description.value_counts().items():
    print(f"{i} {n}")

County Government 1167
Municipality 1157
Trade association 715
No change in address  Just a lobbyist update 681
Advocacy 640
Manufacturing 573
Federal Lobbying Activity 567
University 533
Government training support service company 515
Energy company 456
Municipal government 434
Local Government 429
Develop and carries out policies that advance the wellbeing of dairy producers and coops 363
Energy Company 355
Local government 349
County government 337
County 336
City Government 332
Lead state agency for the promotion of Kansas 295
Automobile manufacturing 283
Trade Association 283
City government 280
Municipal Government 258
Public University 256
trade association 252
Alaska Native Corporation 251
Federation of state water associations 247
Airport 236
Lobbying on behalf of NiSource Inc and issues affecting the company and customers 216
local government 213
State Government Office of the Governor 213
Energy 197
commercial cleaning and sanitation 193
Government Affairs Consulting 189
Tra

In [222]:
ccs_df.loc[
    [isinstance(x, float) for x in ccs_df.clean_client_general_description]
].client_rename.unique()

array(['BASIN ELECTRIC POWER COOPERATIVE', 'RTI INTERNATIONAL',
       'CANADIAN PACIFIC RAILWAY', ..., 'BEAM GLOBAL',
       'PLAINS ALL AMERICAN GP', 'GUSTO'], dtype=object)

In [245]:
government_exclude_description = [
    "affairs",
    "relations",
    "electricity",
    "utility",
    "utilities",
    "sewage",
    "sewerage",
]
government_descriptions = [
    "tribe",
    "town of",
    "village of",
    "government",
    "public agency",
    "Native American Affairs",
    "Economic Development Agency",
    "State District",
    "Transit Authority",
    "Terminal District",
    "metropolitan area",
    "municipality",
    "city",
    "Transportation Authority",
    "Agency providing public transportation",
    "Transit Agency",
    "state department",
    "Public Agency Port District",
    "state agency",
    "The port of",
    "Commuter rail agency",
    "port authority",
    "Regional planning agency",
    "Transportation planning agency",
    "transportation agency",
    "tribal nation",
    "governing body",
    "Commonwealth",
    "quasigovernment",
    "regulatory agency",
    "Public transportation authority",
    "city administration",
    "county",
]

oil_and_gas_description = [
    "oil and gas",
    "oil company",
    "natural gas",
    "exploration and production",
    "petroleum",
]
oil_and_gas_exclude_description = [
    "pipeline",
    "coal",
    "engine",
    "power",
    "electricity",
    "refinery",
    "refining",
    "utility",
    "utilities",
    "petrochemical",
]

In [251]:
org_names = []
for n, row in ccs_df.iterrows():
    phrase = row["clean_client_general_description"].lower()
    client_name = row["client_rename"].lower()
    if (
        (not terms_present(phrase, oil_and_gas_exclude_description, find_any=True))
        and (terms_present(phrase, oil_and_gas_description, find_any=True))
        and (
            not terms_present(
                client_name, oil_and_gas_exclude_description, find_any=True
            )
        )
        and (terms_present(client_name, oil_and_gas_description, find_any=True))
    ):
        org_names.append((row["client_rename"]))
org_list = sorted(list(set(org_names)))
for i in org_list:
    print(f"- {i}")

- AMERICAN EXPLORATION AND PRODUCTION COUNCIL AXPC
- AMERICAN NATURAL GAS ALLIANCE
- AMERICAN PETROLEUM INSTITUTE
- AMERICAS NATURAL GAS ALLIANCE
- ENSTAR NATURAL GAS
- INDEPENDENT PETROLEUM ASSOCIATION OF AMERICA
- INTERSTATE OIL AND GAS COMPACT COMMISSION
- OCCIDENTAL PETROLEUM


In [247]:
ccs_df.loc[
    ccs_df.client_rename.isin(org_list)
].clean_client_general_description.unique()

array(['', 'Oil and gas company', 'Independent oil and gas producer',
       'Energy company', 'Government RelationsLobbying', 'Oil Company',
       'CO2 transportation',
       'Oil and natural gas company specializing in carbon sequestration technology',
       'InvestorOwned Utility',
       'Oil and gas exploration and production company',
       'Natural gas exploration and production company', 'Energy',
       'Production of oil and natural gas',
       'Energy including power plants',
       'Oil and gas exploration and production',
       'Producer of electricity', 'oil and gas exploration',
       'Energy exploration and production', 'oil and gas company',
       'oil and gas exploration and production company',
       'Energy and Petrochemical Company',
       'Independent oil and gas company', 'Energy provider',
       'Energy Company',
       'Development production and exploration of oil and natural gas',
       'Oil and Gas regulation', 'Natural Gas Production',
       'o

In [225]:
sectors_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
sectors = sectors_dict["government"]
for i in sorted(list(set(sectors))):
    print(f"- {i}")

- ABILENE TEXAS
- ABYSSINIAN DEVELOPMENT
- ADAMS COUNTY
- AGUA CALIENTE
- AKUTAN
- ALABAMA STATE PORT AUTHORITY
- ALACHUA COUNTY FLORIDA
- ALAMEDA CO TRANSP COMMISSIONFORMELY ALAMEDA COUNTY TRANSPORTATION IMPROVEMENT
- ALAMEDA COUNTY
- ALBANY
- ALBANY GA
- ALBUQUERQUE
- ALCALDE FAY GOODYEAR AZ
- ALCALDE FAY KEY WEST
- ALCALDE FAY OCOEE FL
- ALCALDE FAY OVIEDO FL
- ALCALDE FAY TREASURE ISLAND
- ALEUTIAN PRIBILOF ISLAND COMMUNITY DEVELOPMENT ASSOCIATION
- ALLEN COUNTY
- AMADOR COUNTY
- AMARILLO TX
- AMERICAN AIRLINES
- AMERICAN FEDERATION OF STATE COUNTY AND MUNICIPAL EMPLOYEES
- AMERICAN PUBLIC WORKS ASSOCIATION
- AMERICANS FOR TAX REFORM
- ANAHEIM
- ANCHORAGE
- ANDERSONIN
- ANOKA COUNTY MINNESOTA
- ARCADIACA
- ARDMORE DEVELOPMENT AUTHORITY
- ARLINGTON
- ARVERNE EAST DEVELOPMENT
- ASPENPITKIN COUNTY AIRPORT BOARD OF COUNTY COMMISSIONERS OF PITKIN COUNTY
- ASSOCIATION OF IDAHO CITIES
- ASSOCIATION OF STATE DAM SAFETY OFFICIALS
- ASSOCIATION OF UNITED STATES ARMY
- ASSOCIATION OF VILLAGE 

In [151]:
sector_descriptions_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sector_company_description_terms.yml"
)
sector_mapping = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
mapped_orgs = []
for v in sector_mapping.values():
    mapped_orgs = mapped_orgs + v

In [152]:
for i, s in enumerate(list(sector_mapping.keys())):
    print(f"{i}. {s}")

0. auto and transportation
1. carbon management services
2. ccs
3. cement
4. business advocacy
5. chemicals
6. clean hydrogen
7. water
8. coal
9. engineering consulting
10. environmental
11. other
12. finance
13. biofuels
14. food and agriculture
15. green hydrogen
16. higher education
17. think tank
18. independent research organization
19. iron and steel
20. labor union
21. government
22. nuclear
23. mining
24. oil and gas
25. oilfield services
26. paper products
27. pipelines
28. power generation and utilities
29. railway
30. renewable energy
31. ads and pr


In [148]:
sectors = {
    "AES": "power generation and utilities",
    "AIR CARRIER ASSOCIATION OF AMERICA": "auto and transportation",
    "AKJ INDUSTRIES": "chemicals",
    "ALASKA AEROSPACE DEVELOPMENT": "auto and transportation",
    "ALLIANCE TO SAVE ENERGY": "think tank",
    "AMENTUM": "engineering consulting",
    "AMERESCO": "renewable energy",
    "AMERI": "other",
    "AMERICAN ASSOCIATION OF STATE HIGHWAY AND TRANSPORTATION OFFICIALS AASHTO": "auto and transportation",
    "AMERICAN BAR ASSOCIATION": "business advocacy",
    "AMERICAN COMMERICAL LINES": "auto and transportation",
    "AMERICAN GREAT LAKES PORTS ASSOCIATION": "auto and transportation",
    "AMERICAN HIGHWAY USERS ALLIANCE": "auto and transportation",
    "AMERICAN INSTITUTE OF ARCHITECTS": "other",
    "AMERICAN INSTITUTE OF PHYSICS": "higher education",
    "AMERICAN INTERNATIONAL": "finance",
    "AMERICAN INVESTMENT COUNCIL": "finance",
    "AMERICAN ISRAEL PUBLIC AFFAIRS COMMITTEE": "other",
    "AMERICAN NETWORK OF COMMUNITY OPTIONS AND RESOURCES": "other",
    "AMERICAN PATRIOT ALTERNATIVE FUELS": "biofuels",
    "AMERICAN PUBLIC WORKS ASSOCIATION": "government",
    "AMERICAN RIVERS": "environmental",
    "AMERICAN ROAD TRANSPORTATION BUILDERS ASSOCIATION": "auto and transportation",
    "AMERICAN SHORT LINE REGIONAL RAILROAD ASSOCIATION": "railway",
    "AMERICAN SMALL MANUFACTURERS COALITION": "business advocacy",
    "AMERICAN SOCIETY OF CIVIL ENGINEERS": "engineering consulting",
    "AMERICAN SUPERCONDUCTOR": "power generation and utilities",
    "AMERICAN TRUCKING ASSOCIATIONS": "auto and transportation",
    "AMERICAN WATERWAYS OPERATORS": "water",
    "AMERICANS FOR AMERICAN ENERGY": "think tank",
    "AMERICANS FOR DEMOCRATIC ACTION": "other",
    "AMERIDREAM": "other",
    "AMERIPRISE FINANCIAL": "finance",
    "AMERIQUAL": "food and agriculture",
    "AMSTED INDUSTRIES": "other",
    "AMT ASSOCIATION FOR MANUFACTURING TECHNOLOGY": "other",
    "AMYRIS BIOTECHNOLOGIES": "biofuels",
    "ANCHORAGE": "government",
    "ANHEUSERBUSCH COMPANIES": "food and agriculture",
    "ANTARES WALTER WHEELER DRIVE SPE": "other",
    "APPLIED MATERIALS": "other",
    "ARAMARK DESTINATIONS": "other",
    "ARCADIACA": "government",
    "ARDMORE DEVELOPMENT AUTHORITY": "government",
    "ARLINGTON CHAMBER OF COMMERCE": "business advocacy",
    "ARTBA": "auto and transportation",
    "ARVERNE EAST DEVELOPMENT": "government",
    "ASHLAND": "chemicals",
    "ASPHALT ROOFING MANUFACTURERS ASSOCIATION": "other",
    "ASSOCIATION OF EQUIPMENT MANUFACTURERS": "auto and transportation",
    "ASSOCIATION OF GLOBAL AUTOMAKERS": "auto and transportation",
    "ASSOCIATION OF LARGE DISTRIBUTION COOPERATIVES": "other",
    "ASSOCIATION OF PROCUREMENT TECHNICAL ASSISTANCE CENTERS": "business advocacy",
    "ASSOCIATION OF STATE DAM SAFETY OFFICIALS": "government",
    "ASSOCIATION OF UNITED STATES ARMY": "government",
    "ASSURANT": "finance",
    "ASTAR AIR CARGO": "auto and transportation",
    "ATC MANAGEMENT": "power generation and utilities",
    "AUBURN": "government",
    "AUSTIN ENERGY": "power generation and utilities",
    "BAE SYSTEMS": "engineering consulting",
    "BAKER COMMODITIES": "food and agriculture",
    "BALANCE ECOSYSTEM TECHNOLOGIES": "other",
    "BARRICK GOLD": "mining",
    "BAY AREA RAPID TRANSIT DISTRICT": "railway",
    "BEAM": "food and agriculture",
    "BEN FRANKLIN TECHNOLOGY PARTNERS": "finance",
    "BETTER PLACE": "environmental",
    "BG": "oil and gas",
    "BINGHAM MCCUTCHEN": "other",
    "BJ": "other",
    "BLACKSTONE": "finance",
    "BLOOM": "clean hydrogen",
    "BLUE DIAMOND GROWERS": "food and agriculture",
    "BOEING": "auto and transportation",
    "BOLLINGER SHIPYARDS": "auto and transportation",
    "BOSTON MASS": "government",
    "BRICK INDUSTRY ASSOCIATION": "cement",
    "BROADROCK RENEWABLES": "renewable energy",
    "BROOKFIELD POWER US ASSET MANAGEMENT": "finance",
    "BUSINESS AND PROFESSIONAL WOMENUSA": "business advocacy",
    "BUSINESS COUNCIL OF NEW ORLEANS": "business advocacy",
    "CADBURY ADAMS": "food and agriculture",
    "CALIFORNIA CITRUS MUTUAL": "food and agriculture",
    "CALIFORNIA PUBLIC EMPLOYEES RETIREMENT SYSTEM": "government",
    "CALIFORNIA STATE ASSOCIATION OF COUNTIES": "government",
    "CALIFORNIA STATE COASTAL CONSERVANCY": "government",
    "CALIFORNIA STATE DEPARTMENT OF JUSTICE": "government",
    "CAMBRIA COMMUNITY DISTRICT CALIFORNIA": "government",
    "CAMP DRESSER MCKEE": "engineering consulting",
    "CANADIAN NATIONAL RAILWAY": "railway",
    "CANADIANAMERICAN BUSINESS COUNCIL": "business advocacy",
    "CAPSTONE TURBINE": "clean hydrogen",
    "CARGILL": "food and agriculture",
    "CARLYLE": "finance",
    "CARMAX": "auto and transportation",
    "CENTER FOR INNOVATIVE TECHNOLOGY": "independent research organization",
    "CHARLES SCHWAB": "finance",
    "CHELAN PUBLIC UTILITY DISTRICT": "power generation and utilities",
    "CHICAGO CLIMATE EXCHANGE": "carbon management services",
    "CITADEL": "finance",
    "CITY PUBLIC SERVICE": "power generation and utilities",
    "CLAIBORNE ELECTRIC COOPERATIVE": "power generation and utilities",
    "CLEAN WATER": "water",
    "CLIMATEMASTER": "renewable energy",
    "CME": "finance",
    "CNOOC": "oil and gas",
}
new_dict = {}
for x in list(sectors.values()):
    new_dict[x] = []
for k, v in sectors.items():
    new_dict[v].append(k)
from utils.io import dict_to_yaml

dict_to_yaml(new_dict, "tmp.yml")

In [ ]:
{
    "COALITION FOR 21ST CENTURY PATENT REFORM": "business advocacy",
    "COALITION FOR AMERICAS GATEWAYS AND TRADE CORRIDORS": "auto and transportation",
    "COALITION FOR AUTO REPAIR EQUALITY": "auto and transportation",
    "COALITION FOR EMISSION REDUCTION PROJECTS": "carbon management services",
    "COALITION FOR FAIR TRANSMISSION POLICY": "power generation and utilities",
    "COALITION OF EPSCOR STATES": "government",
    "COALITION OF URBAN SERVING UNIVERSITIES": "higher education",
    "COALITIONS FOR AMERICA": "business advocacy",
    "COBALT TECHNOLOGIES": "biofuels",
    "COBB ELECTRIC MEMBERSHIP": "power generation and utilities",
    "COCACOLA": "food and agriculture",
    "CODA HOLDINGS": "auto and transportation",
    "COLD SPRING HARBOR LABORATORY": "independent research organization",
    "COLONIAL BANC": "finance",
    "COLUMBUS LEARNING CENTER MANAGEMENT": "higher education",
    "COMMERCIAL SPACEFLIGHT FEDERATION": "auto and transportation",
    "COMMITTEE ON PIPE AND TUBE IMPORTS": "iron and steel",
    "COMMONWEALTH OF KENTUCKY": "government",
    "COMMONWEALTH OF PENNSYLVANIA": "government",
    "COMMUNITIES AGAINST REGIONAL INTERCONNECT": "business advocacy",
    "COMMUNITY ASSOCIATIONS INSTITUTE": "business advocacy",
    "COMMUNITY DEVELOPMENT": "other",
    "COMMUNITY OF NORTHWEST INDIANA": "other",
    "COMMUNITY STREETCAR COALITION": "auto and transportation",
    "COMPETE": "business advocacy",
    "COMPTON": "government",
    "COMPUTING TECHNOLOGY INDUSTRY ASSOCIATION COMPTIA": "other",
    "CONFERENCE OF PROVINCIALS OF": "other",
    "CONGRESSIONAL FIRE INSTITUTE": "government",
    "CONNECTED NATION DEVELOPMENT": "other",
    "CONSOLIDATED EDISON OF NEW YORK": "power generation and utilities",
    "CONSORTIUM OF SOCIAL SCIENCE ASSOCIATIONS": "higher education",
    "CONSTRUCTION INDUSTRY ROUND TABLE": "other",
    "CONSTRUCTION MANAGEMENT ASSOCIATION OF AMERICA": "engineering consulting",
    "CONSUMER TECHNOLOGY ASSOCIATION": "other",
    "CONSUMERS UNITED FOR RAIL EQUITY": "railway",
    "CONTINENTAL AIRLINES": "auto and transportation",
    "CONTINENTAL RESOURCES": "oil and gas",
    "CONVERGYS": "other",
    "CONWAY": "other",
    "COOPER INDUSTRIES": "other",
    "COPPER AND BRASS FABRICATORS COUNCIL": "iron and steel",
    "CORNING": "chemicals",
    "CORPORATION FOR ENTERPRISE DEVELOPMENT": "finance",
    "CORPS NETWORK": "government",
    "CORRECTIONS": "other",
    "COSTA PACIFIC COMMUNITIES": "other",
    "COUNCIL FOR A STRONG AMERICA": "business advocacy",
    "COUNCIL FOR CITIZENS AGAINST GOVERNMENT WASTE": "business advocacy",
    "COUNCIL FOR EXCEPTIONAL CHIDLREN": "higher education",
    "COUNCIL OF ATHABASCAN TRIBAL GOVERNMENTS": "government",
    "COUNCIL OF ENERGY RESOURCE TRIBES": "government",
    "COUNCIL OF STATE AND TERRITORIAL EPIDEMIOLOGISTS": "government",
    "COUNCIL OF THE": "other",
    "COUNCIL ON EQUITABLE REGULATION AND TAXATION": "government",
    "COUNCIL ON S": "other",
    "COUNCIL ON UNDERGRADUATE RESEARCH": "higher education",
    "COUNCIL TREE INVESTORS": "finance",
    "COVIDIEN": "chemicals",
    "CP DEVELOPMENT CO": "other",
    "CRAY": "other",
    "CREATIVE COALITION": "other",
    "CREE": "clean hydrogen",
    "CRH AMERICAS MATERIALS": "cement",
    "CROHNS COLITIS": "other",
    "CROSS MATCH TECHNOLOGIES": "other",
    "CROW TRIBE": "government",
    "CTRAN": "railway",
    "CTUIR": "government",
    "CURRENT": "renewable energy",
    "CYGNUS TOWERS": "other",
    "CYTEC": "chemicals",
    "DAIKIN INDUSTRIES": "chemicals",
    "DAIRYLEA COOPERATIVE": "food and agriculture",
    "DAKOTA CREEK INDUSTRIES": "auto and transportation",
    "DALLAS AREA RAPID TRANSIT DART": "railway",
    "DANFOSS": "chemicals",
    "DANONE": "food and agriculture",
    "DARKNESS TO LIGHT": "other",
    "DAY ZIMMERMAN": "other",
    "DEAN FOODS": "food and agriculture",
    "DECOMMISSIONING PLANT COALITION": "nuclear",
    "DEL MONTE FOODS": "food and agriculture",
    "DELL": "other",
    "DELL TECHNOLOGIES": "other",
    "DELOITTE": "other",
    "DELPHI TECHNOLOGIES": "auto and transportation",
    "DERECKTOR SHIPYARDS": "auto and transportation",
    "DIRECT MARKETING ASSOCIATION": "business advocacy",
    "DISNEY WORLDWIDE": "other",
    "DISTRICT OF COLUMBIA PRIMARY CARE ASSOCIATION": "government",
    "DMB ASSOCIATES": "other",
    "DOCTORS": "other",
    "DOSSIA CONSORTIUM": "other"
    "DOSSIA SERVICE": "other",
    "EADS": "auto and transportation",
    "EAST TENNESSEE ECONOMIC COUNCIL": "business advocacy",
    "EAST TEXAS ELECTRICAL COOPERATIVE": "power generation and utilities",
    "EASTMAN KODAK": "other",
    "EATON": "other",
    "EBAY": "other",
    "ECO POWER SOLUTIONS": "renewable energy",
    "EDF ACTION FUND": "carbon management services",
    "EDISON LEARNING": "higher education",
    "EDISON MATERIALS TECHNOLOGY CENTER": "clean hydrogen",
    "EDISONLEARNING": "higher education",
    "EFFICIENCY FIRST": "renewable energy",
    "EL PASO": "oil and gas",
    "ELECTRIC DRIVE TRANSPORTATION ASSOCIATION": "auto and transportation",
    "ELECTRICAL DISTRICT NO 3": "power generation and utilities",
    "ELECTRICAL DISTRICT NO 8": "power generation and utilities",
    "ELECTRICITIES OF NC": "power generation and utilities",
    "ELECTRONIC DATA SYSTEMS": "other",
    "ELEVANTA": "other",
    "ELK VALLEY RANCHERIA": "government",
    "ELSAG": "other",
    "ELSTER ELECTRICITY": "power generation and utilities",
    "EMC": "other",
    "EMCORE": "other",
    "EMERSON": "other",
    "EMPLOYERS COUNCIL ON FLEXIBLE COMPENSATION": "business advocacy",
    "EMPRESAS FONALLEDAS": "other",
    "ENERGY CONVERSION DEVICES": "renewable energy",
    "ENERGY RESEARCH COALITION": "carbon management services",
    "ENERGYSOLUTIONS": "nuclear",
    "ENRON": "oil and gas",
    "ENSCO": "oilfield services",
    "ENTERPRISE COMMUNITY PARTNERS": "finance",
    "ENTERPRISE HOLDINGS": "auto and transportation",
    "ENTERPRISE MOBILITY": "auto and transportation",
    "ENTERPRISE RENTACAR": "auto and transportation",
    "EQUIPMENT LEASING AND FINANCE ASSOCIATION": "finance",
    "ERISA INDUSTRY COMMITTEE": "business advocacy",
    "ESRI": "other",
    "ESSENTIAL MINERALS ASSOCIATION": "mining",
    "EV CAR RENTAL": "auto and transportation",
    "EVERGLADES TRUST": "environmental",
    "EVERGREEN ELECTROMOTIVE": "auto and transportation",
    "EXELIS": "auto and transportation",
    "EXELON-CONSTELLATION-PEPCO CO": "power generation and utilities",
    "EXPERIAN": "finance",
    "EXPLORATION": "oil and gas",
    "EXPLORATORIUM": "higher education",
    "FAIRPLEX": "other",
    "FARMERS": "food and agriculture",
    "FCA US": "auto and transportation",
    "FEDERAL SIGNAL": "auto and transportation",
    "FEDERATION FOR AMERICAN IMMIGRATION REFORM": "business advocacy",
    "FEDERATION OF EXCHANGE ACCOMMODATORS": "finance",
    "FEDERATION OF STATE HUMANITIES COUNCILS": "higher education",
    "FEEDING AMERICA": "food and agriculture",
    "FEI": "finance",
    "FIELD ALIGNMENT AND TRAINING": "other",
    "FINANCIAL FORUM": "finance",
    "FINSECA": "finance",
    "FIRETRACE AEROSPACE": "auto and transportation",
    "FIRST AMERICA": "finance",
    "FIRST HORIZON NATIONAL": "finance",
    "FIRSTFLOOR K12 SOLUTIONS": "higher education",
    "FISERV": "finance",
    "FISHERS": "other",
    "FLORIDA CITRUS MUTUAL": "food and agriculture",
    "FLORIDA CRYSTALS": "food and agriculture",
    "FLORIDA INLAND NAVIGATION DISTRICT": "auto and transportation",
    "FLORIDA MEP": "business advocacy",
    "FLUOR": "engineering consulting",
    "FM POLICY FOCUS": "business advocacy",
    "FMR": "finance",
    "FOOD MARKETPLACE": "food and agriculture",
    "FOOD RESEARCH AND ACTION CENTER": "food and agriculture",
    "FOOTHILL TRANSIT": "auto and transportation",
    "FORTUNE BRANDS": "chemicals",
    "FOUR RIVERS ECONOMIC DEVELOPMENT": "business advocacy",
    "FRIENDS COMMITTEE ON NATIONAL LEGISLATION": "government",
    "FRIENDS OF CITY PARK": "environmental",
    "FRIENDS OF OBSERVATORY": "environmental",
    "FRIENDS OF SBA MICROLOAN FUND": "finance",
    "FUGRO EARTHDATA AND OTHER FUGRO OPERATING COMPANIES": "engineering consulting",
    "GALENA": "finance",
    "GARRISON DIVERSION CONSERVANCY DISTRICT": "environmental",
    "GEN DIGITAL": "other",
    "GENERAL FEDERATION OF WOMENS CLUBS": "business advocacy",
    "GENERATORS FOR CLEAN AIR COALITION": "carbon management services",
    "GENPOWER": "renewable energy",
    "GEOOPTICS": "independent research organization",
    "GHFB LLP": "finance",
    "GLEN COVENY": "other",
    "GLEN RAVEN 1831 NORTH PARK AVENUE GLEN RAVEN NC 27217": "other",
    "GLOBAL AUTOMAKERS": "auto and transportation",
    "GLOBAL TRAFFIC TECHNOLOGIES": "auto and transportation"
        "GLOBE METALLURGICAL": "iron and steel",
    "GLSEN": "business advocacy",
    "GOLDEN HORIZONS": "other",
    "GOLDEN LIVING": "other",
    "GOVERNMENT ACCOUNTABILITY PROJECT": "government",
    "GOVERNMENT OF UNITED STATES VIRGIN ISLANDS": "government",
    "GREAT RIVER ENERGY": "power generation and utilities",
    "GREATER CLEVELAND REGIONAL TRANSIT AUTHORITY": "railway",
    "GREATER RICHMOND TRANSIT": "railway",
    "GREATER SPOKANE": "business advocacy",
    "GREEN FOR ALL": "environmental",
    "GREEN TECH ACTION FUND": "carbon management services",
    "GREENHOUSE GAS": "carbon management services",
    "GRESSEL ADVISORS": "finance",
    "GROWERS EXPRESS": "food and agriculture",
    "GRTC TRANSIT SYSTEM": "railway",
    "GUARDIAN INDUSTRIES": "iron and steel",
    "H R BLOCK MANAGEMENT": "finance",
    "HARBOR POINT HOLDING": "other",
    "HARRIS": "auto and transportation",
    "HAWAIIAN ELECTRIC": "power generation and utilities",
    "HAYES": "other",
    "HAZELDEN": "other",
    "HDR": "engineering consulting",
    "HERZOG": "engineering consulting",
    "HIGHMARK": "finance",
    "HILLROM": "other",
    "HILLWOOD DEVELOPMENT": "other",
    "HITACHI AMERICA": "auto and transportation",
    "HOME DEPOT": "other",
    "HOUSING ACTION RESOURCE TRUST HART": "finance",
    "HOVNANIAN ENTERPRISES": "other",
    "HR BLOCK MANAGEMENT": "finance",
    "HSBC TECHNOLOGY": "finance",
    "HUBBELL": "other",
    "HUDSON TECHNOLOGIES": "environmental",
    "HUNTER DOUGLAS": "other",
    "HUNTER FAN": "other",
    "HUNTINGTON BANCSHARES": "finance",
    "HYRADIX": "clean hydrogen",
    "IKBI": "other",
    "ILLINOIS TOOL WORKS": "other",
    "IMAGETREE": "other",
    "INCLINE VILLAGE GENERAL IMPROVEMENT DISTRICT": "government",
    "INDEPENDENT FUEL TERMINAL OPERATORS ASSOCIATION": "oil and gas",
    "INDEPENDENT SECTOR": "business advocacy",
    "INDUSTRIAL FASTENERS INSTITUTE": "iron and steel",
    "INDYNE": "other",
    "INFINEON TECHNOLOGIES": "other",
    "INFORMATION TECHNOLOGY INDUSTRY COUNCIL": "other",
    "INGRAM BARGE": "auto and transportation",
    "INLAND VALLEY DEVELOPMENT AGENCY": "government",
    "INNOVATIVE FEDERAL STRATEGIES": "business advocacy",
    "INSCAP": "finance",
    "INSIGHTS ASSOCIATION": "other",
    "INSPERITY": "other",
    "INSPIRE BRANDS": "other",
    "INSTITUTE OF SCRAP RECYCLING INDUSTRIES": "iron and steel",
    "INTEGRATED BUILDING CONSTRUCTION SOLUTIONS": "engineering consulting",
    "INTERGEN": "power generation and utilities",
    "INTERNATIONAL AIR TRANSPORT ASSOCIATION": "auto and transportation",
    "INTERNATIONAL BUSINESS MACHINES": "other",
    "INTERNATIONAL CLIMATE CHANGE PARTNERSHIP": "carbon management services",
    "INTERNATIONAL EMISSIONS TRADING ASSOCIATION": "carbon management services",
    "INTERNATIONAL FOODSERVICE DISTRIBUTORS ASSOCIATION": "food and agriculture",
    "INTERNATIONAL FRANCHISE ASSOCIATION": "business advocacy",
    "INTERNATIONAL FRESH PRODUCE ASSOCIATION": "food and agriculture",
    "INTERNATIONAL POWER AMERICA": "power generation and utilities",
    "INTERNATIONAL SLEEP PRODUCTS ASSOCIATION": "other",
    "INTERNATIONAL SPEEDWAY": "auto and transportation",
    "INTERNATIONAL SWAPS AND DERIVATIVES ASSOCIATION": "finance",
    "INTERSYSTEMS": "other",
    "INTERURBAN TRANSIT PARTNERSHIP GRAND RAPIDS": "railway",
    "INVESTMENT INSTITUTE": "finance",
    "INVISTA": "chemicals",
    "IOGEN": "biofuels",
    "IPRO": "independent research organization",
    "ITA PARTNERS": "other",
    "ITRON": "other",
    "ITTEXELIS": "auto and transportation",
    "IVIVI TECHNOLOGIES": "other",
    "JAMES F MCCONNELL ESQ": "business advocacy",
    "JARDEN ZPRODUCTS": "other",
    "JEFFREY S BURUM ENTERPRISES": "other",
    "JEWISH FEDERATION OF METROPOLITAN CHICAGO JFMC": "business advocacy",
    "JEWISH FEDERATIONS OF": "business advocacy",
    "JOHNSON": "chemicals",
    "JOPLIN AREA COALITION": "government",
    "JULIAN ROBERTSON": "finance",
    "KANSAS BIOSCIENCE AUTHORITY": "biofuels",
    "KANSAS CITY BOARD OF PUBLIC UTILITIES": "power generation and utilities",
    "KAR ASSOCIATES": "engineering consulting",
    "KEEP YOUR HOME ALLIANCE": "finance",
    "KELLOGG": "food and agriculture",
    "KIDSAVE WASHINGTON DC": "other",
    "KLEINER PERKINS CAUFIELD BYERS": "finance",
    "KNIK ARM BRIDGE AND TOLL AUTHORITY": "auto and transportation",
    "KNOWLEDGE LEARNING": "higher education",
    "KOPPERS INDUSTRIES": "chemicals",
    "KRIEG DEVAULT ALEXANDER CAPEHART": "business advocacy"
}


In [144]:
ccs_df.loc[ccs_df.client_rename == "CITIZENSHIP TRUST"].client_name.unique()

array(['CITIZENSHIP TRUST'], dtype=object)

In [159]:
new_vals = list(rename_dict.values())
new_cos = sorted(list(set([x for x in new_vals if x not in mapped_orgs])))
for i, n in enumerate(new_cos):
    # if "oil" in n.lower():
    print(f"- {n}")

- CARE ACTION NOW
- CARE COALITION
- CARE CONTINUUM ALLIANCE
- CARSON REDEVELOPMENT AGENCY
- COALITION FOR 21ST CENTURY PATENT REFORM
- COALITION FOR AMERICAS GATEWAYS AND TRADE CORRIDORS
- COALITION FOR AUTO REPAIR EQUALITY
- COALITION FOR EMISSION REDUCTION PROJECTS
- COALITION FOR FAIR TRANSMISSION POLICY
- COALITION OF EPSCOR STATES
- COALITION OF URBAN SERVING UNIVERSITIES
- COALITIONS FOR AMERICA
- COBALT TECHNOLOGIES
- COBB ELECTRIC MEMBERSHIP
- COCACOLA
- CODA HOLDINGS
- COLD SPRING HARBOR LABORATORY
- COLONIAL BANC
- COLUMBUS LEARNING CENTER MANAGEMENT
- COMMERCIAL SPACEFLIGHT FEDERATION
- COMMITTEE ON PIPE AND TUBE IMPORTS
- COMMONWEALTH OF KENTUCKY
- COMMONWEALTH OF PENNSYLVANIA
- COMMUNITIES AGAINST REGIONAL INTERCONNECT
- COMMUNITY ASSOCIATIONS INSTITUTE
- COMMUNITY DEVELOPMENT
- COMMUNITY OF NORTHWEST INDIANA
- COMMUNITY STREETCAR COALITION
- COMPETE
- COMPTON
- COMPUTING TECHNOLOGY INDUSTRY ASSOCIATION COMPTIA
- CONFERENCE OF PROVINCIALS OF
- CONGRESSIONAL FIRE INSTITUTE

In [ ]:
- AFFORD
- AGC AMERICA
- AIRCONDITIONING HEATING REFRIGERATION INSTITUTE
- ALAN MAUK ASSOCIATES LTD PORT OF CORPUS CHRISTI AUTHORITY
- ALLIANCE FOR AFFORDABLE
- ALTICOR
- AMERICAN BAKERS ASSOCIATION
- AMERICAN COUNCIL OF GIFT ANNUNITIES
- AMERICAN HORSE COUNCIL
- AMERICAN JEWISH COMMITTEE
- AMERICAN LAND TITLE ASSOCIATION
- AMERICAN LIBRARY ASSOCIATION
- AMERICAN MAGLINE
- AMERICAN PACIFIC
- AMERICAN SOCIETY OF TRAVEL ADVISORS
- AMERICAN SUGARBEET GROWERS ASSOCIATION
- AMERICANS FOR DEMOCRATIC ACTION
- AMERIDREAM
- ASSOCIATION OF FUNDRAISING PROFESSIONALS
- ASSOCIATION OF MISSOURI ELECTRIC COOPERATIVES
- BECATECH
- BETTER WORLD FUND
- BLUE DIAMOND GROWERS
- BREAKTHROUGH T1D
- BUSINESS AND PROFESSIONAL WOMENUSA
- BUSINESS COUNCIL OF NEW ORLEANS
- CALIFORNIA STATE ASSOCIATION OF COUNTIES
- CHARITABLE ACCORD
- CENTER FOR COMMUNITY CHANGE
- CENTER FOR RESPONSIBLE LENDING
- CITIZENSHIP TRUST
- CLAIBORNE ELECTRIC COOPERATIVE
- CLEAN WATER
- COUNCIL FOR EXCEPTIONAL CHILDREN
- CREDIT SUISSE SECURITIES
- FRIENDS COMMITTEE ON NATIONAL LEGISLATION
- GENERATORS FOR CLEAN AIR COALITION
- GLOBAL FOREX TRADING GFT
- GOLDBERG GODLES WIENER WRIGHT
- GUTTMACHER INSTITUTE
- HUMAN RIGHTS CAMPAIGN
- LEAGUE OF WOMEN VOTERS
- MUNICIPAL CASTINGS ASSOCIATION
- NATIONAL COUNCIL OF JEWISH WOMEN
- NATIONAL COUNCIL OF NONPROFITS
- NATIONAL EMPLOYMENT LAW PROJECT
- NATIONAL EMPLOYMENT LAWYERS ASSOCIATION NELA
- NATIONAL TAXPAYERS UNION
- NATIONAL WOMEN'S LAW CENTER
- NEHEMIAH
- OUNCE OF PREVENTION FUND
- POPULATION ACTION INTERNATIONAL
- REPRESENTATIVE OF GERMAN INDUSTRY AND TRADE
- RETAIL INDUSTRY LEADERS ASSOCIATION
- RICH ZAMBO ESQ
- RW BECKETT
- SCIENCE COALITION
- SWEETWATER AUTHORITY
- VOLUNTEERS OF AMERICA
- WORLDATWORK


In [ ]:
search_term_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/search_term_list.yml"
)

search_terms = search_term_dict["search_term_list"]
probably_ccs = search_term_dict["probably_ccs"]
maybe_ccs = search_term_dict["maybe_ccs"]

terms = []
for t in search_terms:
    if "," in t:
        terms.append(t.replace('"', "").split(","))
    else:
        terms.append([t.replace('"', "")])

terms = [[substitute(t) for t in tt] for tt in terms]

single_terms = []
multiple_terms = []
for x in terms:
    if len(x) == 1:
        single_terms.append(x[0])
    else:
        multiple_terms.append(x)
print(single_terms)
print(multiple_terms)

In [ ]:
ccs_bills = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/ccs_laws.yml"
)["mostly_ccs_provisions"]
ccs_bills = [re.sub(r"[^\w\s]", "", x) for x in ccs_bills]
print(ccs_bills)

In [ ]:
from utils.io import dict_to_yaml

dict_to_yaml(
    {
        "telecom": list(
            df.loc[
                [
                    ("cable" in x)
                    | ("telecom" in x)
                    | ("television" in x)
                    | ("broadband" in x)
                    | ("broadcasting" in x)
                    | ("broadcasting" in n)
                    | ("phone" in x)
                    | ("cellular" in x)
                    | ("internet" in x)
                    | ("telecom" in n)
                    | ("television" in n)
                    | ("phone" in n)
                    | ("internet" in n)
                    | ("media" in x)
                    | ("media" in n)
                    | ("music" in x)
                    | ("music" in n)
                    | (" TV " in x)
                    | (" TV " in n)
                    for x, n in zip(
                        df.clean_client_general_description, df.client_rename
                    )
                ]
            ]
            .client_rename.value_counts()
            .index
        )
    },
    "tmp.yml",
)

In [ ]:
df = ccs_df.copy(deep=True)

In [ ]:
auto_transport_descriptions = [
    "auto",
    "car",
    "transport",
    "engine",
    "vehicle",
    "shipping",
    "cargo",
    "airline",
    "airplane",
]
ccs_company_descriptions = [
    "capture",
    "ccs",
    "ccus",
    "storage of co2",
    "sequestration",
    "greenhouse gas reduction",
]
iron_and_steel_descriptions = [
    "iron",
    "steel",
]
utility_descriptions = [
    "utility",
    "electricity",
    "power company",
    "power and light",
    "power light",
    "utilities",
    "electric",
    "power generation",
    "power provider",
    "energy provider",
    "energy generation",
    "energy production",
    "energy transmission",
    "energy distribution",
    "generation and distribution",
    "electric and natural gas",
    "transmission and distribution",
    "production and distribution",
    "transmission provider",
]
oil_and_gas_descriptions = [
    "oil and gas",
    "natural gas",
    "midstream",
    "drilling",
    "downstream",
    "upstream",
    "fuel",
    "pipeline",
    "refinery",
    "refining",
    "exploration and production",
    "exploration production",
    "E&P",
    "E & P",
    "oil",
    "petroleum",
]
df["transport_company"] = [
    terms_present(
        x,
        auto_transport_descriptions,
    )
    | (n in sector_mapping["auto and transportation"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["steel_company"] = [
    terms_present(
        x,
        iron_and_steel_descriptions,
    )
    | (n in sector_mapping["iron and steel"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["healthcare_company"] = [
    (
        terms_present(
            x,
            healthcare_company_descriptions,
        )
    )
    | (terms_present(n, healthcare_company_descriptions))
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["ccs_company"] = [
    terms_present(
        x,
        ccs_company_descriptions,
    )
    | (n in sector_mapping["ccs"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["utility"] = [
    terms_present(
        x,
        utility_descriptions,
    )
    | (n in sector_mapping["power generation and utilities"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["oilandgas"] = [
    terms_present(
        x,
        oil_and_gas_descriptions,
    )
    | (n in sector_mapping["oil and gas"])
    for x, n in zip(df["clean_client_general_description"], df["client_rename"])
]
df["probably ccs"] = [terms_present(x, probably_ccs) for x in df.clean_description]
df["maybe ccs"] = [
    terms_present(
        x,
        maybe_ccs,
    )
    for x in df.clean_description
]
df["health_care"] = [
    terms_present(x, ["healthcare", "health care", "medicine", "medical"])
    for x in df.clean_description
]
df["eor"] = [
    terms_present(x, ["EOR", "enhanced oil recovery"]) for x in df.clean_description
]
df["sustainable"] = [
    terms_present(x, ["sustainable", "sustainability"]) for x in df.clean_description
]
df["permitting"] = [
    terms_present(x, ["permit", "permitting", "permits"]) for x in df.clean_description
]
df["carbon tax"] = [
    terms_present(x, ["carbon tax", "tax on carbon"]) for x in df.clean_description
]
df["recycle"] = [
    terms_present(x, ["recycle", "recycling"]) for x in df.clean_description
]
df["hydrogen"] = [
    terms_present(
        x, ["hydrogen", "lowcarbon fuel", "low carbon fuel", "fuelcell", "fuel cell"]
    )
    for x in df.clean_description
]
df["nuclear"] = [terms_present(x, ["nuclear", "uranium"]) for x in df.clean_description]
df["carbon_management"] = [
    terms_present(x, ["carbon management", "carbonmanagement"])
    for x in df.clean_description
]
df["clean_energy"] = [terms_present(x, ["clean energy"]) for x in df.clean_description]
df["biofuels"] = [
    terms_present(x, ["ethanol", "bioethanol", "biofuel", "biodiesel"])
    for x in df.clean_description
]
df["climate"] = [terms_present(x, ["climate"]) for x in df.clean_description]
df["energy security"] = [
    terms_present(x, ["energy security"]) for x in df.clean_description
]

df["low_carbon"] = [
    terms_present(x, ["low carbon", "low emission"]) for x in df.clean_description
]
df["direct air"] = [
    terms_present(x, ["directair", "direct air", "air capture"])
    for x in df.clean_description
]
df["tax"] = [terms_present(x, ["tax"]) for x in df.clean_description]
df["carbon_tax"] = [
    terms_present(x, ["carbon tax", "carbontax", "tax on co2", "tax on carbon"])
    for x in df.clean_description
]
df["coal"] = [terms_present(x, ["coal"]) for x in df.clean_description]
df["clean coal"] = [
    terms_present(x, ["clean coal", "cleancoal"]) for x in df.clean_description
]
df["ccs_single"] = [terms_present(x, single_terms) for x in df.clean_description]
df["ccs_double"] = [
    any([terms_present(x, y, find_any=False) for y in multiple_terms])
    for x in df.clean_description
]
df["ccs_description"] = [
    max(sgl, dbl) for sgl, dbl in zip(df["ccs_single"], df["ccs_double"])
]

df["ccs_bills"] = [terms_present(x, ccs_bills) for x in df.clean_description]
df["definitely_ccs"] = [
    1 if (d + b) > 0 else 0 for d, b in zip(df.ccs_description, df.ccs_bills)
]
df["likely_ccs"] = [
    1 if (d + b + p + c) > 0 else 0
    for d, b, p, c in zip(
        df.ccs_description, df.ccs_bills, df["probably ccs"], df.ccs_company
    )
]
df["potentially ccs"] = [
    1 if (d + m + b + p + c) > 0 else 0
    for d, b, m, p, c in zip(
        df.ccs_description,
        df.ccs_bills,
        df["maybe ccs"],
        df["probably ccs"],
        df.ccs_company,
    )
]
df["net"] = [
    terms_present(
        x,
        [
            "negative emissions technology",
            "negative emissions technologies",
            "negative emission technology",
            "negative emission technologies",
        ],
    )
    for x in df.clean_description
]

In [ ]:
df[
    [
        "healthcare_company",
        "transport_company",
        "oilandgas",
        "utility",
        "steel_company",
        "ccs_company",
        "sustainable",
        "ccs",
        "probably ccs",
        "maybe ccs",
        "nuclear",
        "permitting",
        "low_carbon",
        "ccs_description",
        "definitely_ccs",
        "likely_ccs",
        "potentially ccs",
        "tax",
        "carbon_tax",
        "biofuels",
        "direct air",
        "clean coal",
        "clean_energy",
        "carbon_management",
        "hydrogen",
        "recycle",
        "carbon tax",
        "climate",
        "energy security",
        "ccs_bills",
        "health_care",
        "net",
        "eor",
    ]
].sum()

In [ ]:
print(
    len(
        df.loc[
            (df.definitely_ccs == 1)
            | (df.ccs_company == 1)
            | (df["probably ccs"] == 1)
            | ((df.oilandgas == 1) & (df["maybe ccs"] == 1))
            | ((df.utility == 1) & (df["maybe ccs"] == 1))
            | ((df.transport_company == 1) & (df["maybe ccs"] == 1))
            | ((df.steel_company == 1) & (df["maybe ccs"] == 1))
        ]
    )
)

df.loc[
    (df.ccs == 1)
    | (df.ccs_company == 1)
    | (df["probably ccs"] == 1)
    | ((df.oilandgas == 1) & (df["maybe ccs"] == 1))
    | ((df.utility == 1) & (df["maybe ccs"] == 1))
    | ((df.transport_company == 1) & (df["maybe ccs"] == 1))
    | ((df.steel_company == 1) & (df["maybe ccs"] == 1))
][["lobbyist_apportioned_usd", "activity_apportioned_usd"]].sum()

In [ ]:
for i, row in df.loc[
    (df["maybe ccs"] == 1)
    & ((df.oilandgas == 1) | (df.utility == 1))
    & (df.ccs_description == 0)
].iterrows():
    print(f"{row['client_rename']}: {row['clean_description']}")

In [ ]:
for i, row in df.loc[
    (df["probably ccs"] == 1) & (df.oilandgas == 0) & (df.ccs_description == 0)
].iterrows():
    print(f"{row['client_rename']}: {row['clean_description']}")

In [ ]:
# apportion total lobbying dollars spent (on filing) to individual lobbying activities, using two methods
df["activity_apportioned_usd"] = [
    usd / number_lobbying
    for usd, number_lobbying in zip(
        df.dollars_spent_lobbying, df.total_number_lobbying_activities
    )
]
df["lobbyist_apportioned_usd"] = [
    usd * (n_activity_lobbyists / total_lobbyists)
    if total_lobbyists > 0
    else activity_apportioned
    for usd, n_activity_lobbyists, total_lobbyists, activity_apportioned in zip(
        df.dollars_spent_lobbying,
        df.n_lobbyists_for_activity,
        df.total_number_of_lobbyists_on_filing,
        df.activity_apportioned_usd,
    )
]

In [ ]:
df.loc[df["potentially ccs"] == 1][
    ["lobbyist_apportioned_usd", "activity_apportioned_usd"]
].sum()

In [ ]:
sector_assignments = yaml_to_dict("sectors.yml")


all_companies = []
for _, value in sector_assignments.items():
    all_companies = all_companies + value
print(all_companies)

company_sector_dict = {}
for k, vv in sector_assignments.items():
    for v in vv:
        company_sector_dict = company_sector_dict | {v: k}

In [ ]:
def invert_sector_dict(sectors_path) -> Dict[str, str]:
    """reads in the sector assignment yaml to dict; inverts dict s.t. each company is a key"""
    sector_assignments = yaml_to_dict(sectors_path)

    all_companies = []
    for _, value in sector_assignments.items():
        all_companies = all_companies + value
    print(all_companies)

    company_sector_dict = {}
    for k, vv in sector_assignments.items():
        for v in vv:
            company_sector_dict = company_sector_dict | {v: k}

    return company_sector_dict

In [ ]:
company_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors.yml"
)
lumped_sector_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/lumped_sectors.yml"
)["lumped_sector_assignments"]
df["sector"] = df["client_rename"].copy()
df["sector"] = [company_sector_dict[c] for c in df.sector]
df["lumped_sector"] = [lumped_sector_dict[c] for c in df.sector]

In [ ]:
definitely_ccs_by_year_sector_df = (
    df.loc[df["definitely_ccs"] == 1][
        ["filing_year", "lumped_sector", "lobbyist_apportioned_usd"]
    ]
    .groupby(["filing_year", "lumped_sector"])
    .sum()
    .sort_values(by=["filing_year", "lobbyist_apportioned_usd"], ascending=False)
    .reset_index()
)
all_by_year_sector_df = (
    df[["filing_year", "lumped_sector", "lobbyist_apportioned_usd"]]
    .groupby(["filing_year", "lumped_sector"])
    .sum()
    .sort_values(by=["filing_year", "lobbyist_apportioned_usd"], ascending=False)
    .reset_index()
)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import seaborn as sns

In [ ]:
hue_order

In [ ]:
hue_order = sorted(list(set(lumped_sector_dict.values())))

colors = [
    "tab:green",
    "tab:blue",
    "darkorange",
    "tab:purple",
    "magenta",
    "tab:red",
    "crimson",
    "dimgrey",
    "rebeccapurple",
    "teal",
    "darkblue",
]
color_palette = sns.color_palette(colors)
palette = color_palette
fig, ax = plt.subplots(2, 1, figsize=(16, 10))
sns.barplot(
    ax=ax[0],
    data=definitely_ccs_by_year_sector_df,
    x="filing_year",
    y="lobbyist_apportioned_usd",
    hue="lumped_sector",
    palette=color_palette,
    hue_order=hue_order,
)
ax[0].grid()
ax[0].legend(ncol=2)
ax[0].set_xlim([5, 24])

sns.barplot(
    ax=ax[1],
    data=all_by_year_sector_df,
    x="filing_year",
    y="lobbyist_apportioned_usd",
    hue="lumped_sector",
    palette=color_palette,
    hue_order=hue_order,
)
ax[1].grid()
ax[1].legend(ncol=2, loc="center")
ax[1].set_xlim([5, 24])

In [ ]:
for x in df.client_rename.loc[~df.client_rename.isin(all_companies)].unique():
    print(x.replace(",", ""))

In [ ]:
for i, row in df.clean_description.value_counts().items():
    print(i)

In [ ]:
df.client_general_description.fillna("", inplace=True)
df.loc[
    [
        ("ohio" in x.lower())  # | ("e&p" in x.lower())
        # for x in df.client_general_description
        for x in df.client_rename
    ]
].client_rename.unique()

In [ ]:
sectors = {
    "auto and transportation": [],
    "iron and steel": [],
    "oil and gas": [],
    "chemicals": [],
    "oilfield services": [],
    "finance": [],
    "local government": [],
    "coal": [],
    "power generation and utilities": [],
    "food and agriculture": [],
    "environmental": [],
    "higher education": [],
    "renewable energy": [],
    "pipelines": [],
    "carbon management services": [],
    "paper products": [],
    "railway": [],
    "chamber of commerce": [],
    "green hydrogen": [],
    "clean hydrogen": [],
    "independent research organization": [],
    "mining": [],
    "engineering consulting": [],
}

In [ ]:
for i, row in df.client_rename.value_counts().items():
    print(f"{i}")

In [ ]:
df[["lobbyist_apportioned_usd", "client_rename"]].loc[df.ccs == 1].groupby(
    "client_rename"
).sum().sort_values(by="lobbyist_apportioned_usd", ascending=False).head(60)

In [ ]:
greenwashing_df.filing_id.nunique()

In [ ]:
greenwashing_df.loc[greenwashing_df.ccs == 1].groupby(
    "filing_id"
).first().reset_index().groupby("who_is_lobbying")["dollars_spent_lobbying"].sum()

In [ ]:
xom_entities = [x for x in df.columns if x in entities]
tmp = df[xom_entities].sum()
subset_entities = list(tmp[tmp > 0].index)
greenwashing_df = df.loc[df.ccs == 1].copy()
greenwashing_df[subset_entities].sum()

In [ ]:
search_string = '"carbon capture"OR"capture of carbon"OR"capture of co2"OR"CCUS"OR"CCS"OR"storage of carbon dioxide"OR"storage of co2"OR"capture and sequestration of carbon dioxide"OR"capture and sequestration of co2"OR"blue hydrogen"OR"hydrogen hub"OR"clean hydrogen"OR"45Q"OR"45V"OR"inflation reduction act"OR"117-169"OR"117-58"'

In [ ]:
query_filings = f"https://lda.senate.gov/api/v1/filings/?filing_specific_lobbying_issues={search_string}&filing_period=year_end"

f = requests.get(query_filings)

### Info about congress.gov search for bills with ccs relevance

In [ ]:
#query for congress.gov search
https://www.congress.gov/u/ISk7JcnkfFMeJIdw4sMwB

# "carbon capture" OR "capture of carbon" OR "CCUS" OR "CCS" OR "storage of carbon dioxide" OR "storage of co2" OR "sequestration of carbon dioxide" OR "sequestration of co2" OR "blue hydrogen" OR "clean hydrogen" OR "45Q" OR "45V"

In [ ]:
xom_word_cloud = [
    x.replace("Discussions related to", "")
    .replace("provisions", "")
    .replace("Provisions", "")
    for x in df.description.to_list()
]
xom_word_cloud = [
    x.replace("related", "")
    .replace("Related", "")
    .replace("issues", "")
    .replace("provisions related", "")
    for x in xom_word_cloud
]

In [ ]:
from wordcloud import WordCloud

word_cloud = WordCloud(background_color="white")
word_cloud.generate(" ".join(xom_word_cloud))
import os
from PIL import Image
import numpy as np
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt

plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
df[
    [
        "lobbyist_registrant_name",
        "client_name",
        "who_is_lobbying",
        "general_issue_code",
        "description",
    ]
]

In [ ]:
query_congress = f"https://api.congress.gov/v3/bill/118/hr/1262/summaries?format=json&api_key={data_gov_api_key}"
s = requests.get(query_congress)
s.json()["summaries"][-1]["text"]

In [ ]:
def parse_dollars_spent(income, expense):
    if (income is None) & (expense is None):
        return "income and expense are none", 0.0
    if income is None:
        return "corporation lobbying for itself", float(expense)
    if expense is None:
        return "hired lobbying firm", float(income)
    else:
        return "both income and expense > $0", float(income) + float(expense)


def initialize_row(entity_df, result, filing_id):
    # set up row dictionary using entity booleans
    initialize_row_dict = dict(
        zip(
            [x.lower() for x in list(entity_df["name"])],
            [0] * len(list(entity_df["name"])),
        )
    )
    (
        initialize_row_dict["who_is_lobbying"],
        initialize_row_dict["dollars_spent_lobbying"],
    ) = parse_dollars_spent(result["income"], result["expenses"])
    initialize_row_dict["filing_id"] = filing_id
    initialize_row_dict["url"] = result["url"]
    initialize_row_dict["filing_year"] = int(result["filing_year"])
    initialize_row_dict["filing_period"] = result["filing_period"]
    initialize_row_dict["lobbyist_posted_by_name"] = result["posted_by_name"]

    initialize_row_dict["lobbyist_registrant_id"] = result["registrant"]["id"]
    initialize_row_dict["lobbyist_registrant_name"] = result["registrant"]["name"]
    initialize_row_dict["lobbyist_registrant_contact"] = result["registrant"][
        "contact_name"
    ]
    initialize_row_dict["client_id"] = result["client"]["id"]
    initialize_row_dict["client_client_id"] = result["client"]["client_id"]
    initialize_row_dict["client_name"] = result["client"]["name"]
    initialize_row_dict["affiliated_organizations_present"] = False
    if len(result["affiliated_organizations"]) > 0:
        initialize_row_dict["affiliated_organizations_present"] = True
    initialize_row_dict["convictions_present"] = False
    if len(result["conviction_disclosures"]) > 0:
        initialize_row_dict["convictions_present"] = True
    return initialize_row_dict

In [ ]:
def parse_lobbyists(df, lobbyists, details):
    lobby_dict = {}
    lobby_dict["firm_name"] = details["lobbyist_registrant_name"]
    lobby_dict["client_name"] = details["client_name"]
    lobby_dict["general_issue_code"] = details["general_issue_code"]
    lobby_dict["description"] = details["description"]
    lobby_dict["filing_period"] = details["filing_period"]
    lobby_dict["filing_year"] = details["filing_year"]
    lobby_dict["url"] = details["url"]
    lobby_dict["filing_id"] = details["filing_id"]

    lobby_list = []
    # unpack lobbyists list
    for lobbyist in lobbyists:
        lobby_dict["name"] = (
            lobbyist["lobbyist"]["last_name"]
            + ", "
            + lobbyist["lobbyist"]["first_name"]
        )
        lobby_dict["covered_position"] = "None"
        if "covered_position" in lobbyist:
            lobby_dict["covered_position"] = lobbyist["covered_position"]
        lobby_dict["id"] = lobbyist["lobbyist"]["id"]
        lobby_list.append(lobby_dict.copy())

    if df is None:
        df = pd.DataFrame(
            columns=[
                "name",
                "firm_name",
                "client_name",
                "general_issue_code",
                "description",
                "covered_position",
                "filing_year",
                "filing_period",
                "url",
            ]
        )

    df = pd.concat([df, pd.DataFrame(lobby_list)])
    return df

In [ ]:
# get govt entity names
entity_df = pd.DataFrame(government_entities)

corporations = [
    "Exxon",
    "ExxonMobil",
]  # 'exxon' alone includes ExxonMobil Exxon Mobil Corp, Exxon Mobil Corporation, etc. ['Exxon Mobil'.upper(),'ExxonMobil'.upper(),'Exxon'

row_list = (
    []
)  # initialize holder for each row (which corresponds to a single lobbying activity)
max_page = 20

filing_id = 0
lobbyists_df = None
for corporation in corporations:
    for year in list(range(1999, 2025)):
        page = 1
        while page < max_page:
            # print(f"Querying page {page} for year {year}")
            query_filings = f'https://lda.senate.gov/api/v1/filings/?client_name="{corporation}"&filing_period=year_end&filing_year={year}&page={page}'

            f = requests.get(query_filings)

            if "detail" in f.json():
                break

            results = f.json()["results"]
            page += 1  # increase the page for the next query
            # print(f"{len(results)} filings found")
            if len(results) < 25:
                page = max_page

            for result in results:
                row_dict_base = initialize_row(entity_df, result, filing_id)
                activities = result["lobbying_activities"]

                for activity_count, activity in enumerate(activities):
                    row_dict = row_dict_base.copy()
                    # set up row dictionary using entity booleans
                    row_dict["activity_count"] = activity_count
                    row_dict["general_issue_code"] = activity["general_issue_code"]
                    row_dict["description"] = activity["description"]
                    lobbyists_df = parse_lobbyists(
                        lobbyists_df, activity["lobbyists"], row_dict
                    )

                    lobbyist_id_list = []
                    for lobbyist in activity["lobbyists"]:
                        lobbyist_id_list.append(lobbyist["lobbyist"]["id"])

                    row_dict["lobbyist_ids"] = "; ".join(
                        ["None" if x is None else str(x) for x in lobbyist_id_list]
                    )
                    for entity in activity["government_entities"]:
                        row_dict[entity["name"].lower()] = 1

                    row_list.append(row_dict.copy())

                    row_dict.clear()
                filing_id += 1

tmp = pd.DataFrame(row_list)
entities_influenced = tmp[[x.lower() for x in list(entity_df["name"])]].sum()
zeroed = list(entities_influenced[entities_influenced == 0].index)
xom_df = tmp[[x for x in tmp.columns.values if x not in zeroed]]
xom_df = xom_df.drop_duplicates(subset=xom_df.columns.difference(["filing_id"]))
xom_unique_filing_ids = list(
    xom_df.filing_id.unique()
)  # keep a list of the non-duplicate filing ids

# keep only right company aliases
company_aliases = [
    "EXXON MOBIL CORP",
    "EXXONMOBIL CHEMICAL CO",
    "EXXON USA",
    "EXXON CO USA",
    "EXXON MOBIL CORPORATION",
    "EXXON MOBIL PRODUCTION CO",
    "EXXON MOBIL",
    "EXXONMOBIL CORPORATION",
    "EXXONMOBIL CORP",
    "EXXON MOBILE",
    "EXXONMOBIL GLOBAL SERVICES COMPANY",
    "DCI GROUP, LLC, ON BEHALF OF EXXONMOBIL",
    "EXXONMOBIL",
    "HUNTON ANDREWS KURTH LLP (FORMERLY HUNTON & WILLIAMS LLP BEHALF OF EXXON MOBIL)",
    "EXXON MOBIL COPORATION",
    "EXXON MOBIL CORP.",
]
xom_df = xom_df.loc[xom_df.client_name.isin(company_aliases)]


lobbyists_df = lobbyists_df.loc[
    (lobbyists_df.client_name.isin(company_aliases))
    & (lobbyists_df.filing_id.isin(xom_unique_filing_ids))
]

xom_df["which_congress"] = [which_congress(y) for y in xom_df["filing_year"]]

In [ ]:
test_df[["which_congress", "description"]].loc[
    [x != "" for x in test_df.description]
].to_clipboard()

In [ ]:
# how much money has exxonmobil spent lobbying congress in 25 years?
xom_df[["dollars_spent_lobbying", "filing_id"]].groupby("filing_id").first()[
    "dollars_spent_lobbying"
].sum()

In [ ]:
xom_df.to_csv("xom_lda_filings.csv")
lobbyists_df.to_csv("xom_lobbyists.csv")

In [ ]:
tmp = (
    xom_df[
        [
            "dollars_spent_lobbying",
            "lobbyist_registrant_name",
            "filing_id",
            "filing_year",
        ]
    ]
    .groupby(["lobbyist_registrant_name", "filing_id"])
    .first()
    .reset_index()
)

In [ ]:
tmp.info()

In [ ]:
total_dollars_spent_df = (
    tmp.groupby("lobbyist_registrant_name")
    .agg(
        {
            "dollars_spent_lobbying": "sum",
            "filing_id": "count",
            "filing_year": "min",
            "filing_year": "min",
        }
    )
    .sort_values(by="dollars_spent_lobbying", ascending=False)
)
total_dollars_spent_df.columns = [
    "dollars_spent_lobbying",
    "total_lda_filings",
    "earliest_filing_year",
]
max_year_df = (
    tmp[["lobbyist_registrant_name", "filing_year"]]
    .groupby("lobbyist_registrant_name")
    .agg({"filing_year": "max"})
)
max_year_df.columns = ["latest_filing_year"]
total_dollars_spent_df = total_dollars_spent_df.merge(
    max_year_df, right_index=True, left_index=True
)
total_dollars_spent_df

In [ ]:
xom_dci_lobbyists = list(
    lobbyists_df.loc[lobbyists_df.firm_name == "DCI GROUP, L.L.C."].name.unique()
)

In [ ]:
counts_df = (
    lobbyists_df[["name", "firm_name"]]
    .groupby("firm_name")
    .count()
    .sort_values(by="name")
)
counts_df.columns = ["firm_total_filings"]
df = lobbyists_df.merge(counts_df, left_on="firm_name", right_index=True)
counts_df = (
    lobbyists_df[["name", "firm_name"]]
    .groupby("name")
    .count()
    .sort_values(by="firm_name")
)
counts_df.columns = ["total_number_of_filings"]
df = df.merge(counts_df, left_on="name", right_index=True)

In [ ]:
lobbyists_df[["name", "firm_name", "client_name", "total_number_of_filings"]].groupby(
    "name"
).first().sort_values(by="total_number_of_filings", ascending=False).head(50)

In [ ]:
lob1 = requests.get(
    'https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=1'
)
lob2 = requests.get(
    'https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=2'
)
dci_lobbyists = lob1.json()["results"] + lob2.json()["results"]

In [ ]:
all_dci_lobbyists = []
for dci in dci_lobbyists:
    all_dci_lobbyists.append(dci["last_name"] + ", " + dci["first_name"])

sorted(all_dci_lobbyists)

In [ ]:
xom_df.loc[xom_df.f]

In [ ]:
xom_dci_lobbyists

In [ ]:
xom_dict={}
for x in xom:
    xom_dict['year']=int(x['filing_year'])
    xom_dict['rest']=

In [ ]:
lob = requests.get("https://lda.senate.gov/api/v1/lobbyists/")
filing_types = requests.get(
    "https://lda.senate.gov/api/v1/constants/filing/filingtypes/"
)
general_issues = requests.get(
    "https://lda.senate.gov/api/v1/constants/filing/lobbyingactivityissues/"
)
govt_entities = requests.get(
    "https://lda.senate.gov/api/v1/constants/filing/governmententities/"
)
countries = requests.get("https://lda.senate.gov/api/v1/constants/general/countries/")
states = requests.get("https://lda.senate.gov/api/v1/constants/general/states/")
prefixes = requests.get("https://lda.senate.gov/api/v1/constants/lobbyist/prefixes/")
suffixes = requests.get("https://lda.senate.gov/api/v1/constants/lobbyist/suffixes/")
contribution_items = requests.get(
    "https://lda.senate.gov/api/v1/constants/contribution/itemtypes/"
)

In [ ]:
lob = requests.get(
    'https://lda.senate.gov/api/v1/lobbyists/?registrant_name="DCI GROUP, L.L.C."&page=2'
)
lob.json()["results"]

In [ ]:
contributions = []
for page in range(1, 4):
    c = requests.get(
        f'https://lda.senate.gov/api/v1/contributions/?registrant_name="DCI GROUP, L.L.C."&filing_type=YY&page={page}'
    )
    contributions = contributions + c.json()["results"]

In [ ]:
row_list = []

for dci in contributions:
    row_dict = {}
    for i in dci["contribution_items"]:
        row_dict["contribution_type"] = i["contribution_type"]
        row_dict["contributor_name"] = i["contributor_name"]
        row_dict["payee_name"] = i["payee_name"]
        row_dict["honoree_name"] = i["honoree_name"]
        row_dict["usd"] = float(i["amount"])
        row_dict["date"] = i["date"]
        row_dict["filing_year"] = dci["filing_year"]
        row_dict["registrant_name"] = dci["registrant"]["name"]
        row_list.append(row_dict.copy())
dci_contributions_df = pd.DataFrame(row_list)

In [ ]:
def strip_terms(x, terms):
    for term in terms:
        x = x.replace(term, "")

    return x.split(" (")[0].title()

In [ ]:
terms = [
    "Senator ",
    "US ",
    "Rep. ",
    "U.S. ",
    "Representative ",
    "Sen. ",
    "Congressional ",
    "Candidate ",
    "Congressman ",
    "for ",
    "Friends of ",
]

print(dci_contributions_df.honoree_name.nunique())
print(dci_contributions_df.stripped_honoree_name.nunique())

In [ ]:
dci_contributions_df.honoree_name.unique()
terms = [
    "Senator ",
    "US ",
    "Rep. ",
    "Congresswoman ",
    "U.S. ",
    " 08",
    " Jr.",
    "Jr",
    "Resident Commissioner of Puerto Rico ",
    "Senate ",
    "Campaign",
    "Presidential ",
    "candidate ",
    "Candidate ",
    "Representative ",
    "Sen. ",
    "House ",
    "Congressional ",
    "Candidate ",
    "Representative-Elect ",
    "Campaign Committee",
    "Committee",
    "Congressman ",
    "for ",
    "Friends of ",
    " Senate",
    " Congress",
    "Friend of ",
    "- In-Kind Donation",
]
dci_contributions_df["stripped_honoree_name"] = [
    strip_terms(x, terms) for x in dci_contributions_df.honoree_name
]
dci_contributions_df.stripped_honoree_name.unique()

In [ ]:
yy = list(dci_contributions_df.stripped_honoree_name.unique())
dci_contributions_df["stripped_honoree_name"] = [
    x.strip() for x in dci_contributions_df["stripped_honoree_name"]
]
dci_contributions_df["stripped_honoree_name_sim"] = dci_contributions_df[
    "stripped_honoree_name"
]
for y in yy:
    dci_contributions_df["stripped_honoree_name_sim"] = [
        y if x in y else x for x in dci_contributions_df.stripped_honoree_name_sim
    ]
yy = list(dci_contributions_df.stripped_honoree_name_sim.unique())
for y in yy:
    dci_contributions_df["stripped_honoree_name_sim"] = [
        y if x in y else x for x in dci_contributions_df.stripped_honoree_name_sim
    ]

In [ ]:
yy = list(dci_contributions_df.stripped_honoree_name_sim.unique())
for term, replacement in {
    "Allan": "Alan",
    "Jim": "James",
    "Gordan": "Gordon",
    "&": "And",
    "Georgians": "Johnny",
    "M.": "Mary",
    "Pete": "Peter",
}.items():
    dci_contributions_df["stripped_honoree_name_sim"] = [
        x.replace(term, replacement)
        for x in dci_contributions_df.stripped_honoree_name_sim
    ]

In [ ]:
dci_contributions_df[["stripped_honoree_name_sim", "usd"]].groupby(
    "stripped_honoree_name_sim"
).sum().sort_values(by="usd", ascending=False).sum()

In [ ]:
import pandas as pd
from name_matching.name_matcher import NameMatcher

# define a dataset with bank names
df_a = pd.DataFrame({"recipient1": y})

# alter each of the bank names a bit to test the matching
df_b = pd.DataFrame({"recipient2": sorted(y)})

# initialise the name matcher
matcher = NameMatcher(
    number_of_matches=5, legal_suffixes=True, common_words=False, top_n=50, verbose=True
)

# adjust the distance metrics to use
matcher.set_distance_metrics(["bag", "typo", "refined_soundex"])

# load the data to which the names should be matched
matcher.load_and_process_master_data(
    column="recipient1", df_matching_data=df_a, transform=True
)

# perform the name matching on the data you want matched
matches = matcher.match_names(to_be_matched=df_b, column_matching="recipient2")

# combine the datasets based on the matches
# combined = pd.merge(df_a, matches, how='left', left_index=True, right_on='match_index')
# combined#combined = pd.merge(combined, df_b, how='left', left_index=True, right_index=True)

In [ ]:
# matches.drop(['match_name_0','score_0'],axis=1,inplace=True)
matches

In [ ]:
dci_contributions_df[["filing_year", "usd"]].groupby("filing_year").sum()

In [ ]:
firms = []
# lobbyists=lobbyists.json()['results']
for lobby in lobbyists:
    print(lobby)
    firms.append(lobby["registrant"]["name"])

In [ ]:
firms

In [ ]:
payee = "Friends of Max Baucus".replace(" ", "%20")
year = "2008"
contributor = "Crossroads Strategies, LLC".replace(" ", "%20")
year = "2023"
query_all_contributions = f"https://lda.senate.gov/api/v1/contributions/?contribution_payee={payee}&filing_year={year}"
c = requests.get(query_all_contributions)
baucus = c.json()["results"]

In [ ]:
contributor = "Crossroads Strategies".replace(" ", "%20")
year = "2023"
query_all_contributions = f"https://lda.senate.gov/api/v1/contributions/?contribution_contributor={contributor}&filing_year={year}"
c = requests.get(query_all_contributions)
crossroads = c.json()["results"]

In [ ]:
crossroads

In [ ]:
total = 0

for cc in baucus:
    contributions = cc["contribution_items"]
    for i in contributions:
        if i["payee_name"].lower() == "friends of max baucus":
            print(f'{i["contributor_name"]}, amount: ${i["amount"]}')
            total = total + float(i["amount"])

In [ ]:
total

In [ ]:
govt_entities_df = pd.DataFrame(govt_entities.json())
contribution_items_df = pd.DataFrame(contribution_items.json())
general_issues_df = pd.DataFrame(general_issues.json())
filing_types_df = pd.DataFrame(filing_types.json())

In [ ]:
constants = requests.get("https://lda.senate.gov/api/v1/constants/")
constants.json()

In [ ]:
r.json()

In [ ]:
rr = requests.get(r.json()["filings"] + "?client_name=exxon%20mobil&filing_year=2023")

In [ ]:
rr.json()

In [ ]:
rr.json()["results"][0].keys()

dictionary = rr.json()["results"][0]
dictionary = flatten(dictionary)

for key, value in dictionary.items():
    print(f"{key}: {value}")

In [ ]:
flatten(dictionary)

In [ ]:
query = f"https://developer.nrel.gov/api/pvwatts/v8.json?api_key={config['apikey']}&lat={lat}&lon={lon}&system_capacity={config['kw']}&azimuth={azimuth}&tilt={tilt}&array_type=1&module_type=1&losses=10"
                response = requests.get(query, timeout=20)
                info_dict["ac_annual"] = response.json()["outputs"]["ac_annual"]